In [1]:
import numpy as np
import pandas as pd

## Importar biblioteca de XML Tree

In [2]:
import xml.etree.ElementTree as ET
import nltk
import string


### Abrir arquivo de consultas e colocar flags no ínicio e fim do arquivo

In [3]:
file = open("Consultas.txt" ,"r", encoding='latin-1')

In [4]:
text3 = file.read().replace('\n', '')

In [5]:
text4 = "<tag>"+text3+"</tag>"

### Criar uma tabela com os elementos recuperados para consulta

In [6]:
root = ET.fromstring(text4)
lista = list()
for top in root.findall('top'):
    title = top.find('PT-title').text
    desc = top.find('PT-desc').text
    num = top.find('num').text
    lista.append((num,title,desc))

In [7]:
consultas = pd.DataFrame(lista, columns=['num', 'title', 'desc'])

In [8]:
consultas

,num,title,desc
0,251,Medicina alternativa,Encontrar documentos sobre tratamentos que em...
1,252,Sistemas de reforma e pensões na Europa,Encontrar documentos sobre os esquemas de pen...
2,253,Países com pena de morte,"Em quais países ainda é usada, ou pelo menos ..."
3,254,Danos provocados por terramotos,Encontrar documentos que descrevam os danos c...
4,255,Viciados na Internet,O uso frequente da Internet produz habituação?
5,256,Doença de Creutzfeldt-Jakob,Encontrar documentos que descrevam os sintoma...
6,257,Limpeza étnica nos Balcãs,Encontrar documentos que relatam incidentes e...
7,258,"Impacto da ""fuga de cérebros""",Encontrar documentos sobre o problema causado...
8,259,Urso de Ouro,Encontrar documentos mencionando filmes galar...
9,260,Legislação anti-tabagista,Encontrar documentos que descrevam leis ou qu...


In [9]:
stopwords = nltk.corpus.stopwords.words('portuguese')

## Importar biblioteca de mysql

In [10]:
import pymysql as ps

### Criar conexão com o banco de dados

In [31]:
db = ps.connect(host='127.0.0.1',port=9306,user='root',passwd='root',charset='latin1',db='')

### Criando cursor

In [32]:
cur = db.cursor()

### Espaço para as Query


In [33]:
select_op = '*, weight()'

In [34]:
index_table = 'teste6'

In [35]:
limit = '100'

In [36]:
ranker = 'bm25'

In [46]:
results = pd.DataFrame()

In [47]:
for index, row in consultas.iterrows():
    string_to_match = row['title']
    string_to_match = ' '.join([word for word in string_to_match.split() if word not in (stopwords)])
    string_to_match = string_to_match.replace(' ', ' | ')
    consulta_integer = row['num']
    qry = 'SELECT ' + select_op + ' FROM ' + index_table + ' WHERE MATCH(' + "'" + string_to_match + "'" +  ') LIMIT ' + limit + ' OPTION ranker=' + ranker
    cur.execute(qry)
    row = cur.fetchall()
    result = pd.DataFrame(list(row), columns=['new_id','date','category','docid', 'docno', 'score'])
    result['consulta'] = consulta_integer
    result['index'] = result.index
    results = results.append(result)

In [48]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4059 entries, 0 to 1
Data columns (total 8 columns):
new_id      4059 non-null object
date        4059 non-null object
category    4059 non-null object
docid       4059 non-null object
docno       4059 non-null object
score       4059 non-null object
consulta    4059 non-null object
index       4059 non-null object
dtypes: object(8)
memory usage: 285.4+ KB


In [49]:
results.head()

,new_id,date,category,docid,docno,score,consulta,index
0,950315114,950315,COTIDIANO,FSP950315-114,FSP950315-114,1637,251,0
1,950205127,950205,MAIS!,FSP950205-127,FSP950205-127,1614,251,1
2,950409099,950409,COTIDIANO,FSP950409-099,FSP950409-099,1609,251,2
3,950303004,950303,OPINIÃO,FSP950303-004,FSP950303-004,1600,251,3
4,950720079,950720,COTIDIANO,FSP950720-079,FSP950720-079,1600,251,4


In [50]:
report = pd.DataFrame()

In [51]:
texto = ""

In [52]:
for index, row in results.iterrows():
    texto = texto + row['consulta'] + ' ' + 'Q0' + ' ' + row['docid'] + ' ' + str(row['index']) + ' ' + str(row['score']) + ' ' + 'Cauã_Matheus' + '\n'

In [53]:
texto

' 251  Q0 FSP950315-114 0 1637 Cauã_Matheus\n 251  Q0 FSP950205-127 1 1614 Cauã_Matheus\n 251  Q0 FSP950409-099 2 1609 Cauã_Matheus\n 251  Q0 FSP950303-004 3 1600 Cauã_Matheus\n 251  Q0 FSP950720-079 4 1600 Cauã_Matheus\n 251  Q0 FSP951010-010 5 1600 Cauã_Matheus\n 251  Q0 FSP951111-086 6 1600 Cauã_Matheus\n 251  Q0 FSP951224-124 7 1600 Cauã_Matheus\n 251  Q0 FSP950121-010 8 1599 Cauã_Matheus\n 251  Q0 FSP950309-010 9 1599 Cauã_Matheus\n 251  Q0 FSP950527-009 10 1599 Cauã_Matheus\n 251  Q0 FSP950116-053 11 1587 Cauã_Matheus\n 251  Q0 FSP950101-009 12 1584 Cauã_Matheus\n 251  Q0 FSP950305-010 13 1584 Cauã_Matheus\n 251  Q0 FSP950426-099 14 1584 Cauã_Matheus\n 251  Q0 FSP950823-013 15 1584 Cauã_Matheus\n 251  Q0 FSP950909-009 16 1584 Cauã_Matheus\n 251  Q0 FSP950914-085 17 1584 Cauã_Matheus\n 251  Q0 FSP950918-114 18 1584 Cauã_Matheus\n 251  Q0 FSP950921-031 19 1584 Cauã_Matheus\n 251  Q0 FSP950924-010 20 1584 Cauã_Matheus\n 251  Q0 FSP950926-011 21 1584 Cauã_Matheus\n 251  Q0 FSP951109-

In [54]:
text_file = open("Output.txt", "w")
text_file.write(texto)
text_file.close()

### Fechamento do Cursor e da Conexão

In [30]:
cur.close();db.close() 